In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow.keras as keras

In [4]:
import joblib

In [6]:
df=yf.download("BTC-USD","2020-01-01","2024-01-31")

[*********************100%%**********************]  1 of 1 completed


In [3]:
final_rmse = 5000

In [7]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095
...,...,...,...,...,...,...
2024-01-26,39936.816406,42209.386719,39825.691406,41816.871094,41816.871094,25598119893
2024-01-27,41815.625000,42195.632812,41431.281250,42120.054688,42120.054688,11422941934
2024-01-28,42126.125000,42797.175781,41696.910156,42035.593750,42035.593750,16858971687


In [1]:
def download_data(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    return df

In [5]:
def prepare_data(df, sequence_length):
    data = df.filter(['Close'])
    df = df.dropna()
    df = df[~df.index.duplicated(keep='last')]
    df = df.values
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []

    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    split_index = int(len(scaled_data) * 0.8)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    
    return X_train, X_test, y_train, y_test, scaler

In [6]:
def build_lstm_model(X_train, units=50, layers=2, activation='tanh', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    for _ in range(layers - 1):
        model.add(LSTM(units, return_sequences=True))
    model.add(LSTM(units*2))
    model.add(Dense(25, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [7]:
def train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers):
    
    global final_rmse
    
    loss_history = keras.callbacks.History()
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'model_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.h5', monitor='val_loss', save_best_only=True)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping, model_checkpoint, loss_history])

    lstm_loss_history = loss_history.history['loss']
    
    lstm_predictions = model.predict(X_test)
    lstm_predictions = scaler.inverse_transform(lstm_predictions)
    
    y_test = y_test.reshape(-1,1)
    y_test = scaler.inverse_transform(y_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, lstm_predictions))
    print(f"Root Mean Squared Error (Testing Dataset): {rmse}")
    
    if rmse <= final_rmse:
        final_rmse = rmse
        joblib.dump(model, "BCH_model_lstm.pkl")
        plt.plot(y_test, label='Actual')
        plt.plot(lstm_predictions, label='Predicted')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.legend()
        plt.savefig(f'Plot_actualvspredicted/BCH_actual_vs_predicted.png')
        plt.close()
    
        plt.figure(figsize=(10, 6))
        plt.plot(loss_history.history['loss'], label='Training Loss')
        plt.plot(loss_history.history['val_loss'], label='Validation Loss')
        plt.title('Epoch Loss Curve')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(f'loss_curve/BCH_loss_curve.png')
        plt.close()
    
    return model, rmse


In [8]:
symbol = 'BCH-USD'
start_date = '2020-01-01'
end_date = '2024-01-31'

In [9]:
epochs_range = [100]
batch_sizes = [4,8]
sequence_lengths = [25,60]
units_range = [50, 100]
layers_range = [2, 3]

In [10]:
df = download_data(symbol, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [11]:
results = []

In [12]:
for epochs in epochs_range:
    for batch_size in batch_sizes:
        for sequence_length in sequence_lengths:
            for units in units_range:
                for layers in layers_range:
                    print(f"Training model with epochs={epochs}, batch_size={batch_size}, sequence_length={sequence_length}, units={units}, layers={layers}")
                    X_train, X_test, y_train, y_test, scaler = prepare_data(df, sequence_length)
                    model = build_lstm_model(X_train, units=units, layers=layers)
                    model, rmse = train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers)
                    results.append({'epochs': epochs, 'batch_size': batch_size, 'sequence_length': sequence_length, 'units': units, 'layers': layers, 'rmse': rmse})

Training model with epochs=100, batch_size=4, sequence_length=25, units=50, layers=2


Epoch 1/100

268/268 [==============================] - ETA: 0s - loss: 0.0053

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 41s 65ms/step - loss: 0.0053 - val_loss: 2.9632e-05
Epoch 2/100
268/268 [==============================] - 11s 41ms/step - loss: 0.0021 - val_loss: 1.2149e-04
Epoch 3/100
268/268 [==============================] - 10s 38ms/step - loss: 0.0015 - val_loss: 1.6894e-05
Epoch 4/100
268/268 [==============================] - 11s 42ms/step - loss: 0.0011 - val_loss: 4.3675e-05
Epoch 5/100
268/268 [==============================] - 11s 41ms/step - loss: 9.1091e-04 - val_loss: 5.0934e-04
Epoch 6/100
268/268 [==============================] - 11s 42ms/step - loss: 0.0010 - val_loss: 5.2088e-04
Epoch 7/100
268/268 [==============================] - 11s 40ms/step - loss: 9.3127e-04 - val_loss: 3.2863e-04
Epoch 8/100
268/268 [==============================] - 11s 43ms/step - loss: 9.4349e-04 - val_loss: 2.6084e-04
Epoch 9/100
268/268 [==============================] - 12s 44ms/step - loss: 0.0011 - val_loss: 1.8511e-04
Epoch 10/100
268/268 [===============

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 13s 49ms/step - loss: 0.0032 - val_loss: 5.2879e-04
Epoch 3/100
268/268 [==============================] - 11s 42ms/step - loss: 0.0021 - val_loss: 1.7757e-05
Epoch 4/100
268/268 [==============================] - 12s 44ms/step - loss: 0.0017 - val_loss: 1.7810e-05
Epoch 5/100
268/268 [==============================] - 12s 44ms/step - loss: 0.0016 - val_loss: 3.4391e-05
Epoch 6/100
268/268 [==============================] - 13s 47ms/step - loss: 0.0013 - val_loss: 7.6222e-05
Epoch 7/100
268/268 [==============================] - 13s 50ms/step - loss: 0.0011 - val_loss: 4.5910e-05
Epoch 8/100
268/268 [==============================] - 13s 49ms/step - loss: 0.0014 - val_loss: 2.8852e-04
Epoch 9/100
268/268 [==============================] - 14s 51ms/step - loss: 9.6644e-04 - val_loss: 0.0011
Epoch 10/100
268/268 [==============================] - 13s 48ms/step - loss: 0.0013 - val_loss: 1.8541e-04
Epoch 11/100
268/268 [==========================

...

134/134 [==============================] - 14s 108ms/step - loss: 0.0030 - val_loss: 5.9471e-04
Epoch 3/100
134/134 [==============================] - 14s 105ms/step - loss: 0.0019 - val_loss: 1.8544e-04
Epoch 4/100
134/134 [==============================] - 14s 104ms/step - loss: 0.0015 - val_loss: 3.7342e-05
Epoch 5/100
134/134 [==============================] - 14s 105ms/step - loss: 0.0014 - val_loss: 6.9640e-05
Epoch 6/100
134/134 [==============================] - 14s 103ms/step - loss: 0.0013 - val_loss: 3.1949e-05
Epoch 7/100
134/134 [==============================] - 13s 100ms/step - loss: 0.0013 - val_loss: 5.3681e-05
Epoch 8/100
134/134 [==============================] - 13s 95ms/step - loss: 0.0012 - val_loss: 2.7652e-04
Epoch 9/100
134/134 [==============================] - 10s 74ms/step - loss: 0.0012 - val_loss: 1.1720e-04
Epoch 10/100
134/134 [==============================] - 10s 73ms/step - loss: 0.0012 - val_loss: 3.9023e-05
Epoch 11/100
134/134 [====================

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 11s 82ms/step - loss: 0.0024 - val_loss: 6.8530e-04
Epoch 3/100
134/134 [==============================] - 11s 82ms/step - loss: 0.0019 - val_loss: 2.8188e-04
Epoch 4/100
134/134 [==============================] - 11s 81ms/step - loss: 0.0015 - val_loss: 4.1366e-04
Epoch 5/100
134/134 [==============================] - 11s 83ms/step - loss: 0.0013 - val_loss: 4.0376e-04
Epoch 6/100
134/134 [==============================] - 11s 83ms/step - loss: 0.0013 - val_loss: 6.0294e-04
Epoch 7/100
134/134 [==============================] - 11s 84ms/step - loss: 0.0012 - val_loss: 1.3296e-04
Epoch 8/100
134/134 [==============================] - 11s 83ms/step - loss: 9.9688e-04 - val_loss: 2.7509e-04
Epoch 9/100
134/134 [==============================] - 11s 84ms/step - loss: 9.3946e-04 - val_loss: 0.0013
Epoch 10/100
134/134 [==============================] - 11s 85ms/step - loss: 0.0012 - val_loss: 3.9396e-05
Epoch 11/100
134/134 [======================

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 14s 103ms/step - loss: 0.0039 - val_loss: 2.1690e-05
Epoch 3/100
134/134 [==============================] - 14s 102ms/step - loss: 0.0026 - val_loss: 2.0781e-04
Epoch 4/100
134/134 [==============================] - 14s 103ms/step - loss: 0.0021 - val_loss: 3.9397e-04
Epoch 5/100
134/134 [==============================] - 14s 102ms/step - loss: 0.0016 - val_loss: 0.0015
Epoch 6/100
134/134 [==============================] - 14s 105ms/step - loss: 0.0014 - val_loss: 6.4957e-04
Epoch 7/100
134/134 [==============================] - 14s 101ms/step - loss: 0.0014 - val_loss: 1.3962e-04
Epoch 8/100
134/134 [==============================] - 14s 103ms/step - loss: 0.0011 - val_loss: 7.1083e-05
Epoch 9/100
134/134 [==============================] - 14s 108ms/step - loss: 0.0014 - val_loss: 5.6904e-04
Epoch 10/100
134/134 [==============================] - 14s 106ms/step - loss: 9.5039e-04 - val_loss: 3.1700e-04
Epoch 11/100
134/134 [=================

In [13]:
for result in results:
    print(result)

{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 11.574812775965095}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 3, 'rmse': 31.053576212427743}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 2, 'rmse': 9.883977529684667}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 3, 'rmse': 10.690908391640852}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 60, 'units': 50, 'layers': 2, 'rmse': 10.94968246688757}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 60, 'units': 50, 'layers': 3, 'rmse': 12.436209035187217}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 60, 'units': 100, 'layers': 2, 'rmse': 10.80414416080001}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 60, 'units': 100, 'layers': 3, 'rmse': 10.944102715682918}
{'epochs': 100, 'batch_size': 8, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 9.577124515604899}
{'epochs':

In [14]:
model = joblib.load("BCH_model_lstm.pkl")

In [17]:
X_train, X_test, y_train, y_test, scaler = prepare_data(df, 25)

In [18]:
y_pred = model.predict(X_test)

9/9 [==============================] - 2s 26ms/step


In [19]:
y_pred = y_pred.reshape(1,-1)
y_test = y_test.reshape(1,-1)
y_pred_scaled = scaler.inverse_transform(y_pred)
y_test_scaled = scaler.inverse_transform(y_test)

In [20]:
np.sqrt(mean_squared_error(y_pred_scaled, y_test_scaled))

9.383760352908787

In [26]:
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [27]:
calculate_mape(y_test_scaled, y_pred_scaled)

2.8139902757183313